# Thor Voice Predictions
Este notebook contiene todo relacionado desde perarar los datos json hasta la prediccion final  
la idea es predecir si un usuario ese dia **se va a conectar o no** en base a los parametros proporcionados

No incluiremos nada relacionado a juegos u horas especificas de conexion este notebook apunta a una primera prediccion simple

## Tarea 1
Vamos a transformar de JSON a CSV

In [6]:
# CONVERT JSON TO CSV!
import pandas as pd
import json

# Leer el archivo JSON
with open('datos_voice_log.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Preparar una lista para almacenar todos los registros
all_records = []

# Recorrer cada fecha y sus eventos correspondientes
for date, events in data.items():
    for event in events:
        # Añadir la fecha al evento
        event['date'] = date
        all_records.append(event)

# Convertir los datos a un DataFrame de pandas
df = pd.DataFrame(all_records)

# Exportar el DataFrame a un archivo CSV
df.to_csv('output#1.csv', index=False, encoding='utf-8')

print("El archivo CSV ha sido creado exitosamente.")



El archivo CSV ha sido creado exitosamente.


## Tarea 2
Filtramos, Ordenamos y Transformamos numericamente los datos
para que nos quede
* event
* username
* day
* month
* year
* hour
* minute
* second


In [11]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('output#1.csv')

# Filtrar los eventos no relacionados con gaming ni mute/unmute
non_gaming_events = df[
    ~df['event'].str.contains('game', na=False) & 
    ~df['event'].str.contains('self-muted', na=False) & 
    ~df['event'].str.contains('self-unmuted', na=False)
]

# Convertir la columna 'timestamp' a datetime
non_gaming_events['timestamp'] = pd.to_datetime(non_gaming_events['timestamp'], format='%a, %d %b %Y %H:%M:%S %Z')

# Crear las nuevas columnas de fecha y hora
non_gaming_events['day'] = non_gaming_events['timestamp'].dt.day
non_gaming_events['month'] = non_gaming_events['timestamp'].dt.month
non_gaming_events['year'] = non_gaming_events['timestamp'].dt.year
non_gaming_events['hour'] = non_gaming_events['timestamp'].dt.hour
non_gaming_events['minute'] = non_gaming_events['timestamp'].dt.minute
non_gaming_events['second'] = non_gaming_events['timestamp'].dt.second

# Diccionario de equivalencia userId a username
user_names = {
    "301206456185913346": "Altoemilius",
    "339950447810969612": "fabimontans",
    "588152515946872883": "DiegoPlusUltra",
    "523684629379547162": "Saborcito",
    "625835283333775360": "Nalesh",
    "868627769465139312": "Jorge",
}

# Asegurarse de que los userId sean tratados como cadenas de texto
non_gaming_events['userId'] = non_gaming_events['userId'].astype(str)

# Reemplazar userId con username
non_gaming_events['username'] = non_gaming_events['userId'].map(user_names)

# Seleccionar las columnas deseadas incluyendo username
final_df = non_gaming_events[['event', 'username', 'day', 'month', 'year', 'hour', 'minute', 'second']]

# Ordenar el DataFrame por las columnas de fecha y hora
final_df = final_df.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'second'])

# Exportar el DataFrame a un nuevo archivo CSV
final_df.to_csv('output#2.csv', index=False)

print("El archivo CSV final ha sido creado exitosamente.")




El archivo CSV final ha sido creado exitosamente.


C:\Users\emili\AppData\Local\Temp\ipykernel_20644\2530407507.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_gaming_events['timestamp'] = pd.to_datetime(non_gaming_events['timestamp'], format='%a, %d %b %Y %H:%M:%S %Z')
C:\Users\emili\AppData\Local\Temp\ipykernel_20644\2530407507.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_gaming_events['day'] = non_gaming_events['timestamp'].dt.day
C:\Users\emili\AppData\Local\Temp\ipykernel_20644\2530407507.py:18: SettingWithCopyWarning: 
A value is 

## Tarea 3
Ultima transformacion de los datos para **cada usuario** dejarlos listos para la funcion de Machine Learning  

La tabla final luce asi:

* daysAway (racha de dias fuera de discord)
* consecutiveJoinedDays (racha de dias en discord)
* freeDay (si es sabado o domingo)
* actualDay
* actualMonth
* actualYear
* joinedToday (si ese dia el usuario entro o no)


In [37]:
import pandas as pd
from datetime import timedelta

# Leer el archivo CSV
df = pd.read_csv('output#2.csv')

# Ordenar por usuario y las columnas de fecha y hora
df = df.sort_values(by=['username', 'year', 'month', 'day', 'hour', 'minute', 'second'])

# Función para calcular si es fin de semana
def is_free_day(day, month, year):
    try:
        date = pd.Timestamp(year=year, month=month, day=day)
        return 1 if date.weekday() in [5, 6] else 0
    except ValueError:
        return 0

# Función para calcular days_away
def calculate_days_away(current_date, last_joined_date):
    if last_joined_date:
        return (current_date - last_joined_date).days - 1
    else:
        return None

# Función para calcular consecutive_joined_days
def calculate_consecutive_joined_days(current_date, last_joined_date, last_consecutive_days):
    if last_joined_date and (current_date - last_joined_date).days == 1:
        return last_consecutive_days + 1
    else:
        return 0  # Comienza una nueva racha de días consecutivos
    
# Función para obtener la última hora de conexión
def get_last_connection_time(last_joined_date):
    print("last_date: ", last_joined_date)
    if last_joined_date is not None:
        return last_joined_date.hour, last_joined_date.minute
    else:
        return 0, 0  # Valor por defecto si no hay una última conexión

# Función para procesar los registros de un usuario
def process_user_records(group):
    records = []
    
    # Verificar que todas las fechas sean válidas antes de generar el rango de fechas
    group['valid_date'] = group.apply(
        lambda row: pd.Timestamp(year=row['year'], month=row['month'], day=row['day']), 
        axis=1
    )
    group = group[group['valid_date'].notnull()]

    if group.empty:
        return records
    
    # Crear una lista completa de fechas desde la primera hasta la última conexión
    start_date = group['valid_date'].min()
    end_date = group['valid_date'].max()
    date_range = pd.date_range(start=start_date, end=end_date)
    
    last_joined_date = None
    consecutive_joined_days = 0  # Variable para contar días consecutivos de conexión
    for current_date in date_range:
        row = group[(group['year'] == current_date.year) & 
                    (group['month'] == current_date.month) & 
                    (group['day'] == current_date.day)]
        
        days_away = calculate_days_away(current_date, last_joined_date)
        consecutive_joined_days = calculate_consecutive_joined_days(current_date, last_joined_date, consecutive_joined_days)
        
        if not row.empty:
            joined_today = 1
            last_joined_date = current_date
        else:
            joined_today = 0
        
        # Determinar si el día actual es sábado o domingo
        free_day = is_free_day(current_date.day, current_date.month, current_date.year)
        
        records.append({
            'daysAway': int(days_away) if days_away is not None else 0,
            'consecutiveJoinedDays': consecutive_joined_days,
            'freeDay': free_day,
            'actualDay': current_date.day,
            'actualMonth': current_date.month,
            'actualYear': current_date.year,
            'joinedToday': joined_today
        })
    
    return records

# Iterar sobre cada usuario y guardar archivos separados
for username, group in df.groupby('username'):
    records = process_user_records(group)
    
    # Crear un DataFrame para el usuario actual
    user_df = pd.DataFrame(records)
    
    # Exportar el DataFrame a un archivo CSV
    user_df.to_csv(f'output_{username}.csv', index=False)

print("Los archivos CSV individuales han sido creados exitosamente.")





Los archivos CSV individuales han sido creados exitosamente.


## Construccion del Modelo

### Balanced Random Forest Classifier ✅
Hay muchos modelos y parametros que podriamos ajustar para cada uno de los usuarios  
pero realmente esto no es un proyecto tan serio sino mas bien un ejercicio desafiante  
para predecir de manera levemente acertada la entrada o no de cada usuario.

Como hay clases desbalanceadas usamos balanced random forest classifier

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

def train_and_evaluate_balanced_rf(username):
    # Leer el archivo CSV
    df = pd.read_csv(f"output_{username}.csv")

    # Separar las características (features) de la variable objetivo (target)
    X = df[['daysAway', 'consecutiveJoinedDays', 'freeDay', 'actualDay', 'actualMonth', 'actualYear']]
    y = df['joinedToday']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar BalancedRandomForestClassifier
    model = BalancedRandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"for user: {username}")
    print(f'Model: Balanced Random Forest')
    print(f'Accuracy: {accuracy:.2f}')
    print('Classification Report:')
    print(report)
    print('-' * 50)
    
    # Guardar el mejor modelo
    joblib.dump(model, f'best_joined_today_predictor_{username}.pkl')

# Llamada a la función con un nombre de usuario específico
train_and_evaluate_balanced_rf('DiegoPlusUltra')
train_and_evaluate_balanced_rf('fabimontans')
train_and_evaluate_balanced_rf('Saborcito')



c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future 

for user: DiegoPlusUltra
Model: Balanced Random Forest
Accuracy: 0.74
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.71      0.67         7
           1       0.82      0.75      0.78        12

    accuracy                           0.74        19
   macro avg       0.72      0.73      0.72        19
weighted avg       0.75      0.74      0.74        19

--------------------------------------------------


c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future 

for user: fabimontans
Model: Balanced Random Forest
Accuracy: 0.58
Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.40      0.33         5
           1       0.75      0.64      0.69        14

    accuracy                           0.58        19
   macro avg       0.52      0.52      0.51        19
weighted avg       0.63      0.58      0.60        19

--------------------------------------------------


c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\emili\miniconda3\envs\thor\Lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future 

for user: Saborcito
Model: Balanced Random Forest
Accuracy: 0.68
Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.67      0.40         3
           1       0.92      0.69      0.79        16

    accuracy                           0.68        19
   macro avg       0.60      0.68      0.59        19
weighted avg       0.82      0.68      0.72        19

--------------------------------------------------


In [37]:
def load_model_and_predict(username, new_data):
    # Cargar el modelo guardado
    model = joblib.load(f'best_joined_today_predictor_{username}.pkl')

    # Asegurarse de que los nuevos datos estén en el mismo formato
    new_data_df = pd.DataFrame([new_data])
    
    # Hacer predicciones
    predictions = model.predict(new_data_df)
    return predictions

# Dato nuevo de ejemplo
new_data = {
    'daysAway': 0,
    'consecutiveJoinedDays': 1,
    'freeDay': 0,
    'actualDay': 12,
    'actualMonth': 6,
    'actualYear': 2024
}

# Llamada a la función para predecir con un dato nuevo
predictions = load_model_and_predict('saborcito', new_data)
print("Prediction for new data:", predictions)

Prediction for new data: [1]
